In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import warnings

# Ignorar advertencias para una salida más limpia
warnings.filterwarnings('ignore')

In [2]:
# --- 1. Cargar y Limpiar el Dataset ---
# Asegúrate de que tu archivo 'hypothyroid.csv' esté en el mismo directorio
# o proporciona la ruta completa al archivo.
try:
    df = pd.read_csv('hypothyroid.csv')
except FileNotFoundError:
    print("Error: 'hypothyroid.csv' no encontrado. Asegúrate de que el archivo esté en la ubicación correcta.")
    exit()

# Reemplazar '?' con el valor nulo estándar de numpy (NaN)
df.replace('?', np.nan, inplace=True)

# Eliminar la columna 'TBG' si existe, ya que suele estar vacía
if 'TBG' in df.columns:
    df.drop('TBG', axis=1, inplace=True)

# Convertir la clase objetivo a formato binario (Enfermo=1, Sano=0)
df['binaryClass'] = df['binaryClass'].map({'P': 1, 'N': 0})

# Identificar y convertir columnas numéricas, manejando errores
numeric_cols_to_convert = ['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI']
for col in numeric_cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertir columnas con valores 't'/'f' a formato binario (1/0)
for column in df.columns:
    if df[column].dtype == 'object':
        # Asegurarse de que todos los valores no nulos sean 't' o 'f'
        if df[column].dropna().isin(['t', 'f']).all():
            df[column] = df[column].map({'t': 1, 'f': 0})


# --- INICIO DE LA MODIFICACIÓN FINAL ---

# Lista completa de columnas a eliminar (laboratorio + historial/tratamiento)
cols_to_drop = [
    # Resultados de laboratorio (los que ya habías quitado)
    'TSH measured', 'TSH',
    'T3 measured', 'T3',
    'TT4 measured', 'TT4',
    'T4U measured', 'T4U',
    'FTI measured', 'FTI',

    # Nuevas columnas de historial y tratamiento (la fuga sutil)
    'on thyroxine',
    'query on thyroxine',
    'on antithyroid medication',
    'thyroid surgery',
    'I131 treatment',
    'query hypothyroid',
    'query hyperthyroid'
]

# Crear el DataFrame final para un modelo predictivo "puro"
X_final = df.drop(['binaryClass'] + cols_to_drop, axis=1, errors='ignore')
y_final = df['binaryClass']

print("\n--- Columnas utilizadas para el modelo final ---")
print(X_final.columns.tolist())

# AHORA, VUELVE A EJECUTAR TODO TU PIPELINE CON ESTOS DATOS FINALES:
# 1. Haz el train_test_split con X_final y y_final
# 2. Ajusta tu preprocesador y aplica SMOTE
# 3. Entrena tu modelo XGBoost regularizado
# 4. Evalúa los resultados que obtengas

# --- FIN DE LA MODIFICACIÓN FINAL ---


print("Forma del dataset después de la limpieza:", df.shape)
print("\n--- Conteo de clases original ---")
print(df['binaryClass'].value_counts())




--- Columnas utilizadas para el modelo final ---
['age', 'sex', 'sick', 'pregnant', 'lithium', 'goitre', 'tumor', 'hypopituitary', 'psych', 'TBG measured', 'referral source']
Forma del dataset después de la limpieza: (3772, 29)

--- Conteo de clases original ---
binaryClass
1    3481
0     291
Name: count, dtype: int64


In [ ]:
df.head()

In [3]:
# --- 2. Preparación de Datos para el Modelo ---
# Separar características (X) y variable objetivo (y)
X = df.drop('binaryClass', axis=1)
y = df['binaryClass']

# Identificar automáticamente las características numéricas y categóricas
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [4]:
# --- 3. Definir el Pipeline de Preprocesamiento ---
# Pipeline para datos numéricos: imputar valores faltantes con la mediana y escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para datos categóricos: imputar valores faltantes con el más frecuente y codificar
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos pipelines en un único preprocesador
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])



In [5]:
# --- 4. Aplicar Preprocesamiento y Balanceo con SMOTE ---
# Aplicar el preprocesador al conjunto de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)

# Aplicar el mismo preprocesador (ya ajustado) al conjunto de prueba
X_test_processed = preprocessor.transform(X_test)

print("\n--- Aplicando SMOTE al conjunto de entrenamiento... ---")
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)

print("\n--- Conteo de clases después de SMOTE ---")
print(pd.Series(y_train_resampled).value_counts())




--- Aplicando SMOTE al conjunto de entrenamiento... ---

--- Conteo de clases después de SMOTE ---
binaryClass
1    2784
0    2784
Name: count, dtype: int64


In [7]:
# --- 5. Entrenar el Modelo XGBoost ---
model_xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,

    # --- Parámetros de Regularización ---
    max_depth=4,          # Limitar la profundidad del árbol
    gamma=0.1,            # Hacer el modelo más conservador
    subsample=0.8,        # Usar el 80% de los datos para cada árbol
    colsample_bytree=0.8, # Usar el 80% de las columnas para cada árbol
    reg_lambda=1          # Regularización L2
)

print("\n--- Entrenando el modelo XGBoost con datos balanceados... ---")
model_xgb.fit(X_train_resampled, y_train_resampled)
print("¡Entrenamiento completado!")


--- Entrenando el modelo XGBoost con datos balanceados... ---
¡Entrenamiento completado!


In [8]:
# --- 6. Evaluar el Rendimiento del Modelo ---
print("\n--- Evaluación del Modelo en el Conjunto de Prueba ---")
y_pred = model_xgb.predict(X_test_processed)

print(f"\nPrecisión (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=['Sano (N)', 'Enfermo (P)']))


--- Evaluación del Modelo en el Conjunto de Prueba ---

Precisión (Accuracy): 0.9974

Matriz de Confusión:
[[ 57   1]
 [  1 696]]

Reporte de Clasificación:
              precision    recall  f1-score   support

    Sano (N)       0.98      0.98      0.98        58
 Enfermo (P)       1.00      1.00      1.00       697

    accuracy                           1.00       755
   macro avg       0.99      0.99      0.99       755
weighted avg       1.00      1.00      1.00       755

